In [14]:
# Example of chaotic migration using dynamical tides operator
import rebound
import reboundx
from matplotlib import pyplot as plt
from reboundx import constants
import matplotlib
import numpy as np
matplotlib.rcParams.update({'font.size': 14})

# intial conditions
a0 = 1.5
e0 = 0.01
R_star = 4.65e-3 # 1 solar radius
R_planet = 1.6 * 4.67e-4 # 1.6 Jupiter radii
m_planet = 0.95e-3 # approximately 1 Jupiter mass

# create simulation
sim = rebound.Simulation()
sim.add(m=1, r = R_star)
sim.add(m=m_planet, r=R_planet, a=a0, e=e0)
sim.add(m=1, a=50, inc=87.5 * np.pi / 180)
#sim.integrator = "trace"
#sim.ri_trace.peri_crit_eta = 0.05
#sim.ri_trace.peri_mode = "FULL_IAS15"
sim.move_to_com()

# load tides_constant_time_lag
rebx = reboundx.Extras(sim)
time_lag = rebx.load_force("tides_constant_time_lag")
rebx.add_force(time_lag)
sim.particles[0].params['tctl_k2'] = 0.03
sim.particles[1].params['tctl_k2'] = 0.25
sim.particles[1].params['tctl_tau'] = 10000 * 2*np.pi / (3600*24*365.25) # 10000 s, incredibly dissipative 

# load gr
gr = rebx.load_force("gr_full")
rebx.add_force(gr)
gr.params["c"] = constants.C


N = 50000
t = np.linspace(0, 2e5, N)

for i, time in enumerate(t):
    o = sim.particles[1].orbit(primary=sim.particles[0])
    
    if i % 50 == 0 and i != 0 or o.e >= 0.96:
        print(time, o.a, o.e)
        
    
    sim.integrate(2 * np.pi * time)
    



200.0040000800016 1.499935594438263 0.009939893586897034
400.0080001600032 1.4999128778331918 0.009951374219139246
600.0120002400048 1.499943183403305 0.009973296291348611
800.0160003200064 1.4999485631574554 0.010065317373680131
1000.0200004000079 1.4999922092207776 0.010128219265662496
1200.0240004800096 1.4999406390645071 0.010237646818257708
1400.0280005600112 1.499907732620649 0.010381230372284608
1600.0320006400127 1.4999040129520118 0.01049546783075695
1800.0360007200143 1.4999343968992258 0.010734873373951296
2000.0400008000158 1.4999521427738642 0.01090531383864171
2200.0440008800174 1.4999408694456733 0.011069399977041814
2400.048000960019 1.4999741539246088 0.011343090028749166
2600.0520010400205 1.4999469966955223 0.011504721722050746
2800.0560011200223 1.499930639324192 0.011804022754092779
3000.0600012000236 1.4998781958811138 0.012070136852623559
3200.0640012800254 1.499934926364038 0.012267283957420506
3400.0680013600268 1.4999542239473216 0.012629925667548347
3600.0720

KeyboardInterrupt: 